## Task/End goal
> Be able to classify CODM users as satsfied/dissatisfied based on their review

In [22]:
import nltk
import pandas as pd

In [2]:
data = pd.read_csv("Call Of Duty.csv")

In [3]:
data.head()

,reviews,ratings
0,I've been playing for years now and it's been ...,4
1,An annoying experience all round. Ever since t...,1
2,"I love the game though. It's close to real, bu...",3
3,I've been playing for years now and it's been ...,4
4,An annoying experience all round. Ever since t...,1


## EDA/Feature engineering

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882 entries, 0 to 3881
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  3882 non-null   object
 1   ratings  3882 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 60.8+ KB


In [5]:
data.isna().sum()

reviews    0
ratings    0
dtype: int64

In [6]:
data.describe()

,ratings
count,3882.000000
mean,3.202473
std,1.572016
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [7]:
# Classifying reviews as binary (Satisfied =1, dissatisfied = 0)
# Satisfied user == 3+ stars rating, dissatisfied == 2< stars rating
data["satisfied"] = data["ratings"].apply(lambda x: 1 if x >=3 else 0)

In [8]:
data.head()

,reviews,ratings,satisfied
0,I've been playing for years now and it's been ...,4,1
1,An annoying experience all round. Ever since t...,1,0
2,"I love the game though. It's close to real, bu...",3,1
3,I've been playing for years now and it's been ...,4,1
4,An annoying experience all round. Ever since t...,1,0


In [20]:
reviews_by_satisfaction = data.groupby("satisfied")

In [35]:
# number of satisfied and dissatisfied users
reviews_by_satisfaction["satisfied"].count()

satisfied
0    1382
1    2500
Name: satisfied, dtype: int64